In [3]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

In [42]:
nameFile = open("features.txt")
nameList = []
for feature in nameFile:
    feature = feature.strip()
    nameList.append(feature)

    
X_train = pd.read_csv("cleaned_X_train.csv", names = nameList)
X_test = pd.read_csv("cleaned_X_test.csv", names = nameList)

X_train = X_train.values
X_test = X_test.values


y_train = pd.read_csv("y_train.csv", names = ["Action"])
y_test = pd.read_csv("y_test.csv", names = ["Action"])
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()







In [43]:
print(y_train.shape)


(7767,)


In [44]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

In [45]:
K = pairwise_kernels(X_train, metric = 'rbf', gamma = .05)
print(K.shape)

(7767, 7767)


In [46]:
lrg_lst = []
sml_lst = []
for i in range(1, 7767):
    cur_k = rbf_kernel(X_train[0], X_train[i], .05)
    if(len(lrg_lst) == 10):
        lrg_lst.sort(reverse = True)
        sml_lst.sort()
    if(len(lrg_lst) < 10):
        lrg_lst.append(cur_k)
        sml_lst.append(cur_k)
    
    else:
        if cur_k > lrg_lst[9]:
            lrg_lst.append(cur_k)
            lrg_lst.sort(reverse = True)
            lrg_lst.pop()
        if cur_k < sml_lst[9]:
            sml_lst.append(cur_k)
            sml_lst.sort()
            sml_lst.pop()
            
print('Maximum similarities: \n', [e for e in lrg_lst])
print('Minimum similarities: \n', [e for e in sml_lst])

Maximum similarities: 
 [0.39009221571106545, 0.332786181248158, 0.3221287097647293, 0.3179946907071938, 0.31422658666043557, 0.31148442099116647, 0.30703010753425686, 0.3062282131938721, 0.30440608482184983, 0.3036318398212423]
Minimum similarities: 
 [1.5416430089433268e-18, 3.4228311243882322e-18, 1.4770790140620293e-14, 1.7541174314854206e-13, 3.8017269485076573e-13, 7.916168974764486e-13, 1.6527249327485234e-12, 1.823933878710029e-12, 6.605868992055673e-12, 1.1854790023685458e-11]


In [47]:
def grad_opt_adapt(grad_func, beta0, nit=7767, lr_init=1e-3):
    beta = beta0
    lr = lr_init
    L,Lgrad = grad_func(beta0)
    for it in range(nit):
        beta1 = beta - lr*Lgrad
        L1, Lgrad1 = grad_func(beta1)
        df_est = Lgrad.T@(beta1-beta)
        alpha = 0.5
        if (L1-L < alpha*df_est) and (L1 < L):
            lr = lr*2
            L = L1
            Lgrad = Lgrad1
            beta = beta1
        else:
            lr = lr/2 
        if (lr < 1e-15):
            break;
    return beta

# note that I'm taking a slightly different approach here to what we did in `lab_grad_descent`.
# happy discuss if there are any questions. The approach here will lead to faster convergence.
def kernel_grad(alpha,K,y,lamb):
    z = K@alpha
    h = 1/(1+np.exp(-z))
    L = np.sum((1-y)*z - np.log(h)) + lamb*np.sum(z*alpha)

    # Gradient
    Lgrad = (h-y) + 2*lamb*alpha
    return L, Lgrad

def log_fit(K,y,lamb,nit=1000):
    """
    Function which minizes the logistic regression loss 
    """
    kernel_grad_eval = lambda alpha: kernel_grad(alpha,K,y,lamb)
    alpha0 = np.zeros(K.shape[0])
    alpha = grad_opt_adapt(kernel_grad_eval, alpha0, nit=nit, lr_init=1e-5)
    return alpha

In [48]:

alpha = log_fit(K, y_train, 0, 7767)
print("alpha is", alpha)

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
D:\anaconda\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


alpha is [7.52708738e+150 7.52708738e+150 7.52708738e+150 ... 1.88177184e+150
 1.88177184e+150 1.88177184e+150]


In [49]:
Ktest = pairwise_kernels(X_test, X_train, metric = 'rbf', gamma = .05)

In [52]:
yhat = (Ktest.dot(alpha))
print(yhat)

acc = np.mean(yhat == y_train)
print("Test accuracy = %f" % acc)

[3.40983624e+153 4.94783924e+153 7.17195179e+153 ... 1.25975625e+153
 1.28127574e+153 1.29980985e+153]
Test accuracy = 0.000000


D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  after removing the cwd from sys.path.
